In [1]:
from typing import Optional
from enum import Enum

from sqlalchemy import (
create_engine, URL, Column, Date, Integer, Numeric, SmallInteger, String, Table,
ForeignKeyConstraint, PrimaryKeyConstraint, select, func
)
from sqlalchemy.orm import sessionmaker, mapped_column, Mapped, declarative_base, relationship
from sqlalchemy.orm.base import Mapped

In [2]:
Base = declarative_base()
metadata = Base.metadata

In [3]:
class Receipt(Base):
    __tablename__ = 'receipt'
    __table_args__ = (
        #ForeignKeyConstraint(['product_cd'], ['product.product_cd'], name='receipt_product_fk'),
        #ForeignKeyConstraint(['store_cd'], ['store.store_cd'], name='receipt_store_fk'),
        PrimaryKeyConstraint('sales_ymd', 'store_cd', 'receipt_no', 'receipt_sub_no', name='receipt_pk'),
    )

    sales_ymd = mapped_column(Integer, nullable=False)
    store_cd = mapped_column(String(6), nullable=False)
    receipt_no = mapped_column(SmallInteger, nullable=False)
    receipt_sub_no = mapped_column(SmallInteger, nullable=False)
    sales_epoch = mapped_column(Integer)
    customer_id = mapped_column(String(14))
    product_cd = mapped_column(String(10))
    quantity = mapped_column(Integer)
    amount = mapped_column(Integer)

    #product: Mapped[Optional['Product']] = relationship('Product', back_populates='receipt')
    #store: Mapped['Store'] = relationship('Store', back_populates='receipt')

    def __str__(self):
        return f'{self.customer_id} bought {self.product_cd} at {self.store_cd}'

In [4]:
url_object = URL.create(
    "postgresql+psycopg",
    username="postgres",
    password="postgres",  # plain (unescaped) text
    host="postgres",
    port="5432",
    database="postgres",
    query={"options": "-c search_path=public"},
)

engine = create_engine(url_object)

SessionLocal = sessionmaker(engine)

In [5]:
class ReceiptColumn(str, Enum):
    sales_ymd = 'sales_ymd'
    store_cd = 'store_cd'
    receipt_no = 'receipt_no'
    receipt_sub_no = 'receipt_sub_no'

In [6]:
class Customer(Base):
    __tablename__ = 'customer'
    __table_args__ = (
        #ForeignKeyConstraint(['application_store_cd'], ['store.store_cd'], name='customer_store_fk'),
        PrimaryKeyConstraint('customer_id', name='customer_pk'),
    )
    
    customer_id = mapped_column(String(14))
    customer_name = mapped_column(String(20))
    gender_cd = mapped_column(String(1))
    gender = mapped_column(String(2))
    birth_day = mapped_column(Date)
    age = mapped_column(Integer)
    postal_cd = mapped_column(String(8))
    address = mapped_column(String(128))
    application_store_cd = mapped_column(String(6))
    application_date = mapped_column(String(8))
    status_cd = mapped_column(String(12))

    def __str__(self):
        return f'{self.customer_name} ({self.birth_day})'

In [7]:
class CustomerColumn(str, Enum):
    birth_day = 'birth_day'

In [8]:
class Store(Base):
    __tablename__ = 'store'
    __table_args__ = (
        PrimaryKeyConstraint('store_cd', name='store_pk'),
    )
    
    store_cd = mapped_column(String(6))
    store_name = mapped_column(String(128))
    prefecture_cd = mapped_column(String(2))
    prefecture = mapped_column(String(5))
    address = mapped_column(String(128))
    address_kana = mapped_column(String(128))
    tel_no = mapped_column(String(20))
    longitude = mapped_column(Numeric)
    latitude = mapped_column(Numeric)
    floor_area = mapped_column(Numeric)

In [9]:
statement = select(Customer).order_by(getattr(Customer, CustomerColumn.birth_day)).limit(10)

with SessionLocal() as session:
    customers = session.scalars(statement).all()

for customer in customers:
    print(customer)

村山 菜々美 (1928-11-26)
吉村 朝陽 (1928-12-14)
熊沢 美里 (1929-01-07)
内村 拓郎 (1929-01-12)
天野 拓郎 (1929-01-15)
鶴田 里穂 (1929-01-28)
山元 美紀 (1929-02-22)
中田 里穂 (1929-04-08)
宇野 南朋 (1929-04-09)
金谷 恵梨香 (1929-04-09)


In [10]:
statement = select(Customer).order_by(getattr(Customer, CustomerColumn.birth_day).desc()).limit(10)

with SessionLocal() as session:
    customers = session.scalars(statement).all()

for customer in customers:
    print(customer)

大村 美里 (2007-11-25)
福山 はじめ (2007-10-02)
柴田 真悠子 (2007-09-17)
松井 京子 (2007-08-09)
山内 遥 (2007-06-03)
小柳 夏希 (2007-04-18)
広末 まなみ (2007-03-30)
長野 美紀 (2007-03-22)
福岡 瞬 (2007-03-10)
矢口 莉緒 (2007-03-05)


In [28]:
statement = select(
    Receipt.customer_id,
    func.count().label('receipt_count'),
    func.avg(Receipt.amount).label('amount_mean'),
    func.avg(Receipt.quantity).label('quantity_min')
).group_by(Receipt.customer_id)

with SessionLocal() as session:
    receipts = session.execute(statement).all()

In [34]:
receipts[0]._asdict()

{'customer_id': 'CS001311000059',
 'receipt_count': 2,
 'amount_mean': Decimal('1151.0000000000000000'),
 'quantity_min': Decimal('1.00000000000000000000')}